In [1]:
using Knet

include("../src/data.jl")
include("../src/model.jl")

train! (generic function with 1 method)

In [2]:
datadir = "../data/enwik8"
jld2dir = "../jld2/enwik8.jld2"

if !isfile(jld2dir)
    BATCHSIZE = 64
    println("Reading data from directory: $datadir")
    println("Setting batch size to $BATCHSIZE")
    vocab = Vocab("$datadir/train.txt")
    trainfile = TextReader("$datadir/train.txt", vocab)
    validfile = TextReader("$datadir/valid.txt", vocab)
    testfile = TextReader("$datadir/test.txt", vocab)
    dtrn = TextData(trainfile, batchsize=BATCHSIZE)
    ddev = TextData(validfile, batchsize=BATCHSIZE)
    dtst = TextData(testfile, batchsize=BATCHSIZE)
    println("Saving data from $jld2dir")
    Knet.save(jld2dir, "dtrn", dtrn, "dtst", dtst, "ddev", ddev)
else 
    println("Loading data from $jld2dir")
    (dtrn, dtst, ddev) = Knet.load(jld2dir, "dtrn", "dtst", "ddev")
    vocab = dtrn.src.vocab
end

Loading data from ../jld2/enwik8.jld2


64

In [3]:
@info "Initializing and Training Language Model"
epochs, em_size, hidden_size, layers = 12, 1024, 1024, 4
println("embedding size: ", em_size)
println("hidden size: ", hidden_size)
println("layers: ", layers)

println("Collecting training data...")
println("epochs: ", epochs)
ctrn = collect(dtrn)
trnx10 = collect(flatten(shuffle!(ctrn) for i in 1:epochs))
trnmini = ctrn[1:20]
dev = collect(ddev)

model = SimpleLSTMModel(em_size, hidden_size, vocab; layers=layers, dropout=0.2)

┌ Info: Initializing and Training Language Model
└ @ Main In[3]:1


embedding size: 1024
hidden size: 1024
layers: 4
epochs: 12


SimpleLSTMModel(Embed(P(KnetArray{Float32,2}(1024,206))), LSTM(input=1024,hidden=1024,layers=4,dropout=0.2), Linear(P(KnetArray{Float32,2}(206,1024))), 0.2, Vocab(Dict("54" => 67,"101" => 4,"41" => 52,"65" => 38,"168" => 126,"159" => 175,"228" => 183,"190" => 117,"227" => 96,"88" => 104…), ["<s>", "<unk>", "32", "101", "116", "97", "105", "111", "110", "114"  …  "210", "239", "211", "198", "212", "240", "205", "220", "222", "200"], 2, 1, split))

In [ ]:
model = train!(model, trnx10, dev, trnmini)


┣                    ┫ [0.01%, 1/16488, 00:49/226:25:51, 49.44s/i] (trnloss = (5.1319246f0,), trnppl = (169.34273f0,), trnbpc = (7.403802212779256,), devloss = 5.1315694f0, devppl = 169.28258f0, devbpc = 7.403289704479685)
┣                    ┫ [0.11%, 18/16488, 02:03/31:11:03, 4.30s/i] (trnloss = (3.5610683f0,), trnppl = (35.200783f0,), trnbpc = (5.13753557153007,), devloss = 3.5333343f0, devppl = 34.237934f0, devbpc = 5.097523807806837)
┣                    ┫ [0.21%, 35/16488, 03:16/25:41:29, 4.34s/i] (trnloss = (3.5498776f0,), trnppl = (34.80906f0,), trnbpc = (5.121390872162975,), devloss = 3.5233138f0, devppl = 33.89657f0, devbpc = 5.083067290140612)
┣                    ┫ [0.32%, 52/16488, 04:31/23:52:04, 4.39s/i] (trnloss = (3.547924f0,), trnppl = (34.741123f0,), trnbpc = (5.118572420480635,), devloss = 3.518152f0, devppl = 33.722054f0, devbpc = 5.075620441358252)
┣                    ┫ [0.42%, 70/16488, 05:45/22:33:34, 4.10s/i] (trnloss = (3.544971f0,), trnppl = (34.63868f0,),

┣▋                   ┫ [3.74%, 617/16488, 45:01/20:02:54, 4.38s/i] (trnloss = (3.1912284f0,), trnppl = (24.318281f0,), trnbpc = (4.603969372221957,), devloss = 3.177541f0, devppl = 23.987696f0, devbpc = 4.58422266821519)
┣▊                   ┫ [3.85%, 634/16488, 46:15/20:02:47, 4.36s/i] (trnloss = (3.1797414f0,), trnppl = (24.040535f0,), trnbpc = (4.587397123984816,), devloss = 3.1677105f0, devppl = 23.75304f0, devbpc = 4.57004029089444)
┣▊                   ┫ [3.95%, 651/16488, 47:28/20:02:19, 4.31s/i] (trnloss = (3.2154942f0,), trnppl = (24.915602f0,), trnbpc = (4.6389774727009865,), devloss = 3.202185f0, devppl = 24.58619f0, devbpc = 4.6197762976627565)
┣▊                   ┫ [4.05%, 668/16488, 48:42/20:02:13, 4.36s/i] (trnloss = (3.1564322f0,), trnppl = (23.486649f0,), trnbpc = (4.55376901229631,), devloss = 3.1461482f0, devppl = 23.246353f0, devbpc = 4.538932412971676)
┣▊                   ┫ [4.15%, 685/16488, 49:56/20:01:59, 4.34s/i] (trnloss = (3.1376014f0,), trnppl = (23.048515

┣█▍                  ┫ [7.48%, 1233/16488, 01:29:15/19:53:26, 4.32s/i] (trnloss = (2.669386f0,), trnppl = (14.431105f0,), trnbpc = (3.851109814625184,), devloss = 2.679348f0, devppl = 14.575586f0, devbpc = 3.865482060792487)
┣█▌                  ┫ [7.58%, 1250/16488, 01:30:29/19:53:28, 4.36s/i] (trnloss = (2.648749f0,), trnppl = (14.136345f0,), trnbpc = (3.8213372100037195,), devloss = 2.659688f0, devppl = 14.291829f0, devbpc = 3.8371186820121927)
┣█▌                  ┫ [7.68%, 1267/16488, 01:31:42/19:53:15, 4.28s/i] (trnloss = (2.635405f0,), trnppl = (13.948961f0,), trnbpc = (3.8020858160314384,), devloss = 2.6458824f0, devppl = 14.095878f0, devbpc = 3.817201371215771)
┣█▌                  ┫ [7.78%, 1283/16488, 01:32:54/19:53:58, 4.55s/i] (trnloss = (2.6332917f0,), trnppl = (13.919514f0,), trnbpc = (3.7990369075969426,), devloss = 2.644843f0, devppl = 14.081236f0, devbpc = 3.815702026465549)
┣█▌                  ┫ [7.89%, 1301/16488, 01:34:09/19:53:13, 4.15s/i] (trnloss = (2.612478f0,

┣██▏                 ┫ [11.08%, 1827/16488, 02:12:14/19:53:18, 4.13s/i] (trnloss = (2.3090749f0,), trnppl = (10.065109f0,), trnbpc = (3.331290876531138,), devloss = 2.3248134f0, devppl = 10.2247715f0, devbpc = 3.353996714028649)
┣██▏                 ┫ [11.18%, 1844/16488, 02:13:27/19:53:12, 4.30s/i] (trnloss = (2.292919f0,), trnppl = (9.903804f0,), trnbpc = (3.307982755790305,), devloss = 2.3085463f0, devppl = 10.05979f0, devbpc = 3.330528305457212)
┣██▎                 ┫ [11.29%, 1861/16488, 02:14:40/19:53:03, 4.28s/i] (trnloss = (2.2842479f0,), trnppl = (9.818298f0,), trnbpc = (3.2954730817318456,), devloss = 2.3012984f0, devppl = 9.987141f0, devbpc = 3.3200717602847534)
┣██▎                 ┫ [11.38%, 1877/16488, 02:15:52/19:53:33, 4.56s/i] (trnloss = (2.280037f0,), trnppl = (9.777041f0,), trnbpc = (3.2893979665727677,), devloss = 2.2971213f0, devppl = 9.945511f0, devbpc = 3.3140454881985204)
┣██▎                 ┫ [11.49%, 1894/16488, 02:17:06/19:53:30, 4.32s/i] (trnloss = (3.33328

┣██▉                 ┫ [14.90%, 2456/16488, 02:57:44/19:53:11, 4.09s/i] (trnloss = (2.2261589f0,), trnppl = (9.264213f0,), trnbpc = (3.2116683437235114,), devloss = 2.243287f0, devppl = 9.424259f0, devbpc = 3.2363791549647813)
┣██▉                 ┫ [15.00%, 2473/16488, 02:58:58/19:53:14, 4.37s/i] (trnloss = (2.210404f0,), trnppl = (9.119399f0,), trnbpc = (3.1889387726201806,), devloss = 2.2276196f0, devppl = 9.277756f0, devbpc = 3.213775819127184)
┣███                 ┫ [15.10%, 2490/16488, 03:00:12/19:53:11, 4.32s/i] (trnloss = (2.2091758f0,), trnppl = (9.108207f0,), trnbpc = (3.187167007350992,), devloss = 2.2250526f0, devppl = 9.253969f0, devbpc = 3.2100723447235042)
┣███                 ┫ [15.21%, 2508/16488, 03:01:26/19:52:44, 4.11s/i] (trnloss = (2.2000027f0,), trnppl = (9.025038f0,), trnbpc = (3.1739329423670988,), devloss = 2.2195256f0, devppl = 9.202964f0, devbpc = 3.202098541098901)
┣███                 ┫ [15.32%, 2526/16488, 03:02:40/19:52:17, 4.11s/i] (trnloss = (2.1899981

...

┣████████▏           ┫ [40.65%, 6703/16488, 08:03:19/19:48:51, 4.30s/i] (trnloss = (1.2802805f0,), trnppl = (3.5976486f0,), trnbpc = (1.8470542862395172,), devloss = 1.2938073f0, devppl = 3.6466439f0, devbpc = 1.866569329615571)
┣████████▏           ┫ [40.75%, 6719/16488, 08:04:31/19:48:59, 4.53s/i] (trnloss = (1.2772784f0,), trnppl = (3.5868645f0,), trnbpc = (1.8427232471428392,), devloss = 1.293134f0, devppl = 3.6441894f0, devbpc = 1.8655979716034978)
┣████████▏           ┫ [40.85%, 6736/16488, 08:05:45/19:48:59, 4.32s/i] (trnloss = (1.2773902f0,), trnppl = (3.5872655f0,), trnbpc = (1.8428845668693483,), devloss = 1.2928644f0, devppl = 3.6432073f0, devbpc = 1.865209118829897)
┣████████▏           ┫ [40.96%, 6753/16488, 08:06:57/19:48:56, 4.27s/i] (trnloss = (1.2757728f0,), trnppl = (3.581468f0,), trnbpc = (1.8405511062624418,), devloss = 1.2904092f0, devppl = 3.6342735f0, devbpc = 1.8616669641527266)
┣████████▏           ┫ [41.06%, 6770/16488, 08:08:11/19:48:57, 4.35s/i] (trnloss = (

┣████████▊           ┫ [44.27%, 7299/16488, 08:46:12/19:48:39, 4.27s/i] (trnloss = (1.2409623f0,), trnppl = (3.4589403f0,), trnbpc = (1.7903301084186913,), devloss = 1.260585f0, devppl = 3.5274844f0, devbpc = 1.8186396566292657)
┣████████▊           ┫ [44.37%, 7316/16488, 08:47:25/19:48:38, 4.30s/i] (trnloss = (1.2388818f0,), trnppl = (3.4517517f0,), trnbpc = (1.7873286671958335,), devloss = 1.2578939f0, devppl = 3.5180044f0, devbpc = 1.8147573202686886)
┣████████▉           ┫ [44.47%, 7333/16488, 08:48:39/19:48:40, 4.37s/i] (trnloss = (1.2369509f0,), trnppl = (3.445093f0,), trnbpc = (1.7845428922171578,), devloss = 1.2559917f0, devppl = 3.5113187f0, devbpc = 1.8120129931088709)
┣████████▉           ┫ [44.58%, 7351/16488, 08:49:54/19:48:33, 4.15s/i] (trnloss = (1.2376323f0,), trnppl = (3.447441f0,), trnbpc = (1.7855259450494896,), devloss = 1.2575423f0, devppl = 3.5167675f0, devbpc = 1.8142499714486433)
┣████████▉           ┫ [44.69%, 7368/16488, 08:51:08/19:48:34, 4.36s/i] (trnloss = 

┣█████████▌          ┫ [47.88%, 7895/16488, 09:29:16/19:48:51, 4.36s/i] (trnloss = (1.2064091f0,), trnppl = (3.3414643f0,), trnbpc = (1.7404804211181537,), devloss = 1.2295617f0, devppl = 3.4197302f0, devbpc = 1.7738825476034266)
┣█████████▌          ┫ [47.99%, 7912/16488, 09:30:29/19:48:51, 4.34s/i] (trnloss = (1.2041615f0,), trnppl = (3.3339624f0,), trnbpc = (1.7372378602187863,), devloss = 1.2270541f0, devppl = 3.4111657f0, devbpc = 1.7702648925425275)
┣█████████▌          ┫ [48.09%, 7929/16488, 09:31:44/19:48:53, 4.38s/i] (trnloss = (1.2051309f0,), trnppl = (3.3371959f0,), trnbpc = (1.7386364231356026,), devloss = 1.2284862f0, devppl = 3.4160542f0, devbpc = 1.7723309201273423)
┣█████████▋          ┫ [48.19%, 7946/16488, 09:32:57/19:48:53, 4.32s/i] (trnloss = (1.2050037f0,), trnppl = (3.3367715f0,), trnbpc = (1.738452917647132,), devloss = 1.2282251f0, devppl = 3.4151626f0, devbpc = 1.7719542781219528)
┣█████████▋          ┫ [48.30%, 7963/16488, 09:34:12/19:48:54, 4.36s/i] (trnloss 

┣██████████▎         ┫ [51.52%, 8494/16488, 10:12:09/19:48:17, 4.36s/i] (trnloss = (1.181401f0,), trnppl = (3.258937f0,), trnbpc = (1.704401384672201,), devloss = 1.2069365f0, devppl = 3.343227f0, devbpc = 1.7412412723655708)
┣██████████▎         ┫ [51.62%, 8511/16488, 10:13:23/19:48:18, 4.35s/i] (trnloss = (1.1793242f0,), trnppl = (3.2521753f0,), trnbpc = (1.7014051029288693,), devloss = 1.2043457f0, devppl = 3.3345766f0, devbpc = 1.7375035734143693)
┣██████████▎         ┫ [51.72%, 8528/16488, 10:14:37/19:48:18, 4.35s/i] (trnloss = (1.1778789f0,), trnppl = (3.2474785f0,), trnbpc = (1.6993199852698087,), devloss = 1.2038643f0, devppl = 3.3329718f0, devbpc = 1.7368091074701852)
┣██████████▎         ┫ [51.83%, 8545/16488, 10:15:51/19:48:18, 4.32s/i] (trnloss = (1.177438f0,), trnppl = (3.2460473f0,), trnbpc = (1.6986839934269178,), devloss = 1.2037907f0, devppl = 3.3327262f0, devbpc = 1.736702822191952)
┣██████████▍         ┫ [51.93%, 8562/16488, 10:17:04/19:48:18, 4.31s/i] (trnloss = (1.

┣███████████         ┫ [55.15%, 9093/16488, 10:54:57/19:47:36, 4.10s/i] (trnloss = (1.1529763f0,), trnppl = (3.1676066f0,), trnbpc = (1.6633931534698696,), devloss = 1.1845189f0, devppl = 3.2691138f0, devbpc = 1.7088995909055178)
┣███████████         ┫ [55.25%, 9110/16488, 10:56:10/19:47:35, 4.29s/i] (trnloss = (1.1520771f0,), trnppl = (3.1647596f0,), trnbpc = (1.6620958883344115,), devloss = 1.1845136f0, devppl = 3.2690964f0, devbpc = 1.708891851686229)
┣███████████         ┫ [55.36%, 9127/16488, 10:57:24/19:47:36, 4.33s/i] (trnloss = (1.1518539f0,), trnppl = (3.1640534f0,), trnbpc = (1.6617739368119961,), devloss = 1.1841909f0, devppl = 3.2680416f0, devbpc = 1.7084262946503435)
┣███████████         ┫ [55.45%, 9143/16488, 10:58:37/19:47:42, 4.55s/i] (trnloss = (1.1505226f0,), trnppl = (3.159844f0,), trnbpc = (1.6598532345671604,), devloss = 1.1831709f0, devppl = 3.26471f0, devbpc = 1.7069548110895612)
┣███████████         ┫ [55.56%, 9160/16488, 10:59:51/19:47:44, 4.37s/i] (trnloss = (

In [ ]:
Knet.save("../jld2/baseline-1024-4.jld2", "model", model)

In [ ]:
testloss = loss(model, dtst)
(testloss=testloss, testppl=exp.(testloss), testbpc=(testloss ./ log(2)))

In [ ]:
devloss = loss(model, ddev)
(devloss=devloss, devppl=exp.(devloss), devbpc=(devloss ./ log(2)))

In [ ]:
s = generate(model, start="Syrian people are", maxlength=1024)